In [ ]:
import os
import numpy
import pandas
import pytube
import datetime
import requests
from json import loads
from pytube import YouTube
from dotenv import load_dotenv
from urllib.parse import quote_plus
from sqlalchemy import create_engine

In [ ]:
f = open("all.txt", "r")
links_to_read = f.read().split('\n')
links_to_read

In [ ]:
def get_description(video: YouTube) -> str:
    i: int = video.watch_html.find('"shortDescription":"')
    desc: str = '"'
    i += 20  # excluding the `"shortDescription":"`
    while True:
        letter = video.watch_html[i]
        desc += letter  # letter can be added in any case
        i += 1
        if letter == '\\':
            desc += video.watch_html[i]
            i += 1
        elif letter == '"':
            break
    return loads(desc)

def get_thumb(name, thumb):
    img_data = requests.get(thumb).content
    with open('./thumb/{0}.png'.format(name), 'wb') as handler:
        handler.write(img_data)


In [ ]:
names, authors, descriptions, ratings, thumbs, titles, nn_views, lengths = [], [], [], [], [], [], [], []
for l in links_to_read:
    name = l[l.index('?v=')+len('?v='):]
    print(name)
    try:
        yy = YouTube(l)
        yy.check_availability()
        author = yy.author
        # description = yy.description
        description = get_description(yy)
        rating = yy.rating
        thumb = yy.thumbnail_url
        get_thumb(name, thumb)
        title = yy.title
        n_views = yy.views
        length = yy.length
        # yy.streams.first().download(output_path='data\\', filename='{0}.mp4'.format(name))
    except Exception as e:
        print(e)
        author = ''
        description = ''
        rating = numpy.nan
        thumb = ''
        title = ''
        n_views = numpy.nan
        length = numpy.nan
    names.append(name)
    authors.append(author)
    descriptions.append(description)
    ratings.append(rating)
    thumbs.append(thumb)
    titles.append(title)
    nn_views.append(n_views)
    lengths.append(length)
thumbnails = ['{0}.png'.format(name) for name in names]
summary = pandas.DataFrame(data={'_name': titles,
                                 '_author': authors,
                                 '_description': descriptions,
                                 '_link': links_to_read,
                                 'thumbnail': thumbnails,
                                 })
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
summary.to_excel('./summaries/summary_{0}.xlsx'.format(now), index=False)
summary

In [ ]:
summary

In [ ]:
load_dotenv()

DB_USER = os.getenv('DB_USER')
DB_PW = os.getenv('DB_PW')
DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')

conn = create_engine("postgresql+psycopg2://{0}:{1}@{2}/{3}".format(
    DB_USER, quote_plus(DB_PW), DB_HOST, DB_NAME
)).connect()

In [ ]:
summary.to_sql(name='pizzas', con=conn, if_exists='append', index=False)